# Resolving the bill

It's time to leave the hotel and pay your bill. Unfortunately, there's been some kind of foul-up in the hotel's admin system, and people's bills have been mixed up. Can you help the staff clarify what's happened?

The bills are complex, but they are just strings of events that are billed. We can (without loss of generality) use letters for each different event, so we can treat the bills as strings of letters.

The set of bills the staff have to work through are given as a bill on each line, with a leading number (an integer) that gives the index of the bill; the index number is separated from the bill by a colon.

For instance, the bills could look like this:

```
0: accbadaadc
1: bbbbaabada
2: cdaacacadcddbccacdab
3: bbcdabbaaabcbcadcaac
4: accbbabbdbaaabadaadc
5: acadcdddab
6: aacccabaddcdaddaabdc
```

You know what you spent, so you've helpfully included your bill as the first line of the file, at index 0. 

You can find your bill in the mixed-up line at index 4, like this:

```
accbbabbdbaaabadaadc
*** **  *   *    ***
```

(the stars indicate the bits that come from your bill). There's more than one way to extract your bill from that mixed-up line, and that's OK: you're just trying to find possible matches at the moment. 

Your bill is also in the line at index 6:

```
aacccabaddcdaddaabdc
 * ** ** *  *   * **
```

but you can't find your bill in any other lines. That means your bill occurs 2 times in this list of bills, if you exclude the line zero (which you added).

## Part 1

Given that your bill is at line 0, and the list of bills as [09-bills.txt](09-bills.txt), **how many _other_ lines contain your bill as a subsequence?**

It seems there's some method behind the madness of the bills. Enough that you have hope of paying the bill and leaving soon enough to catch your flight home!

Somewhere in the list of bills is a line that's a mixture of your bill and your friend's bill. Your bill is still on line 0. Your friend's bill is on line 1. In the example above, you can see that line 4 is the only line that's a mixture of your two bills.

## Part 2
Given that your bill is at line 0, your friend's is on line 1, and the list of bills is still in [09-bills.txt](09-bills.txt), **which line is a mixture of your bill and your friends's bill?** (There's only one such line.)

Given two strings a and b and a target c, could c be formed form some interleaving/merge of a and b?

For example,
Given:
s1 = "aabcc",
s2 = "dbbca",

When s3 = "aadbbcbcac", return true.
When s3 = "aadbbbaccc", return false.

In [1]:
with open('09-bills.txt') as f:
    bills = {int(l.strip().split(': ')[0]): l.strip().split(': ')[1] for l  in f.readlines()}
len(bills)

148

In [2]:
def show_table(table):
    return '\n'.join(
        ' '.join('T' if table[i, j] else '.' for j in sorted(set([k[1] for k in table])))
        for i in sorted(set([k[0] for k in table])))        

In [3]:
def show_annotated_table(table, bps):
    return '\n'.join(' '.join('.' if (i, j) not in bps else bps[i, j][2] if table[i, j] else '.' for j in sorted(set([k[1] for k in table])))
        for i in sorted(set([k[0] for k in table])))

## Part 1

In [4]:
def show_backtrace_s(bps):
    i = max([0] + [k[0] for k in bps])
    j = max([0] + [k[1] for k in bps])
    chars = ''
    if (i, j) in bps:
        while i != 0 and j != 0:
            if bps[i, j][3] == 'seq1':
                chars += bps[i, j][2].upper()
            else:
                chars += bps[i, j][2]
            i, j = bps[i, j][0], bps[i, j][1] 
        return ''.join(list(reversed(chars)))
    else:
        return ''

In [5]:
def is_subseq_recursive(s1, s2):
    if not s1:
        return True
    elif len(s1) > len(s2):
        return False
    else:
        if s1[-1] == s2[-1]:
            return is_subseq_recursive(s1, s2[:-1]) or is_subseq_recursive(s1[:-1], s2[:-1])
        else:
            return is_subseq_recursive(s1, s2[:-1])

In [6]:
def is_subseq(seq1, seq2, return_backpointers=False, return_table=False, debug=False):
    """Return true if seq1 is a subsequence of seq2.
    If return_backpointers, also return the set of backpointers to
    reconstruct the subsequence"""
    
    # dp_table[i, j] is True if first i characters of seq1 can
    # be found in the first j characters of seq2
  
    dp_table = {(i, j): False
               for i in range(len(seq1)+1)
               for j in range(len(seq2)+1)}

    backpointers = {}
    
    for i in range(len(seq1)+1):
        for j in range(i, len(seq2)+1):
            if i == 0:
                dp_table[i, j] = True
                if debug: print('aa', i, j, '!', '!', dp_table[i, j])
            elif j == 0:
                dp_table[i, j] = False
                if debug: print('zz', i, j, '!', '!', dp_table[i, j])
            else:
                # extend by character from s2
                if dp_table[i, j-1]:
                    dp_table[i, j] = True
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                    if debug: print('s2', i, j, seq1[i-1], seq2[j-1], dp_table[i, j])                
                # extend by character from s1
                if dp_table[i-1, j-1] and seq1[i-1] == seq2[j-1]:
                    dp_table[i, j] = True
                    backpointers[i, j] = (i-1, j-1, seq1[i-1], 'seq1')                
                    if debug: print('s1', i, j, seq1[i-1], seq2[j-1], dp_table[i, j])
                if not dp_table[i, j]:
                    if debug: print('xx', i, j, seq1[i-1], seq2[j-1], dp_table[i, j]) 
    
    if return_backpointers or return_table:
        retval = [dp_table[len(seq1), len(seq2)]]
        if return_backpointers:
            retval += [backpointers]
        if return_table:
            retval += [dp_table]
        return tuple(retval)
    else:
        return dp_table[len(seq1), len(seq2)]

In [35]:
def is_subseq_rows(seq1, seq2, return_backpointers=False, debug=False):
    """Return true if seq1 is a subsequence of seq2.
    If return_backpointers, also return the set of backpointers to
    reconstruct the subsequence"""
    
    # dp_table[i, j] is True if first i characters of seq1 can
    # be found in the first j characters of seq2
  
    backpointers = {}
    
    for i in range(len(seq1)+1):
        row = [False] * (len(seq2)+1)
        for j in range(i, len(seq2)+1):
            if i == 0:
                row[j] = True
                if debug: print('aa', i, j, '!', '!', row[j])
            elif j == 0:
                row[j] = False
                if debug: print('zz', i, j, '!', '!', row[j])
            else:
                # extend by character from s2
                if row[j-1]:
                    row[j] = True
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                    if debug: print('s2', i, j, seq1[i-1], seq2[j-1], row[j])                
                # extend by character from s1
                if previous_row[j-1] and seq1[i-1] == seq2[j-1]:
                    row[j] = True
                    backpointers[i, j] = (i-1, j-1, seq1[i-1], 'seq1')                
                    if debug: print('s1', i, j, seq1[i-1], seq2[j-1], row[j])
                if not row[j]:
                    if debug: print('xx', i, j, seq1[i-1], seq2[j-1], row[j])
        previous_row = row
    
    if return_backpointers:
        retval = [row[-1]]
        if return_backpointers:
            retval += [backpointers]
        return tuple(retval)
    else:
        return row[-1]

In [7]:
sum(1 for s in bills
   if s != 0
   if is_subseq(bills[0], bills[s]))

22

In [36]:
sum(1 for s in bills
   if s != 0
   if is_subseq_rows(bills[0], bills[s]))

22

In [8]:
%%timeit
sum(1 for s in bills
   if s != 0
   if is_subseq(bills[0], bills[s]))

1 loop, best of 3: 8.02 s per loop


In [37]:
%%timeit
sum(1 for s in bills
   if s != 0
   if is_subseq_rows(bills[0], bills[s]))

1 loop, best of 3: 3.04 s per loop


In [9]:
[s for s in bills
   if s != 0
   if is_subseq(bills[0], bills[s])]

[11,
 21,
 25,
 30,
 33,
 38,
 43,
 45,
 48,
 50,
 52,
 55,
 56,
 61,
 75,
 80,
 91,
 103,
 104,
 113,
 144,
 146]

In [65]:
%time is_subseq(bills[0], bills[11]), is_subseq(bills[0], bills[3])

CPU times: user 168 ms, sys: 0 ns, total: 168 ms
Wall time: 166 ms


(True, False)

In [73]:
# %time is_subseq_recursive(bills[0], bills[11])

In [39]:
v, bp, t = is_subseq(bills[0], bills[11], return_backpointers=True, return_table=True)
print(show_annotated_table(t, bp))
print(show_backtrace_s(bp))

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. c g d b h c e c a b h g f e g h a a c a c c h d g g g g e e f g h g b c g e g a e c a e c g d g e e e g g d d h b a e h b d f b g g f g f h b f e b d h c c d c g h c f a b h c c c h b h f f c c d 

## Part 2

In [47]:
def is_interleave_recursive(s1, s2, s3):
    if not s1:
        return s2 == s3
    elif not s2:
        return s1 == s3
    else:
        if s1[-1] == s2[-1] and s1[-1] == s3[-1]:
            return is_interleave_recursive(s1[:-1], s2, s3[:-1]) or is_interleave(s1, s2[:-1], s3[:-1])
        elif s1[-1] == s3[-1]:
            return is_interleave_recursive(s1[:-1], s2, s3[:-1])
        elif s2[-1] == s3[-1]:
            return is_interleave(s1, s2[:-1], s3[:-1])
        else:
            return False

In [40]:
def is_interleave(seq1, seq2, seq3, return_backpointers=False, return_table=False, debug=False):
    """Return true if seq3 is some interleaved merge of seq1 and seq2.
    If return_backpointers, also return the set of backpointers to
    reconstruct the interleaving"""
    
    # dp_table[i, j] is True if first i+j characters of seq is made up of 
    # an interleaving of the first i characters of seq1 and the 
    # first j characters of seq2
    
    if len(seq1) + len(seq2) != len(seq3):
        if return_backpointers or return_table:
            retval = [False]
            if return_backpointers:
                retval += [{}]
            if return_table:
                retval += [{}]
            return tuple(retval)
        else:
            return False
    
    dp_table = {(i, j): False
               for i in range(len(seq1)+1)
               for j in range(len(seq2)+1)}

    backpointers = {}

    for i in range(len(seq1)+1):
        for j in range(len(seq2)+1):
            if i == 0 and j == 0:
                dp_table[i, j] = True
                if debug: print('xxxx', i, j, '!', '!', '!', dp_table[i, j])
            elif i == 0:
                # extend by character from seq2
                if dp_table[i, j-1] and seq2[j-1] == seq3[i+j-1]:
                    dp_table[i, j] = True
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                if debug: print('seq2', i, j, '!', seq2[j-1], seq3[i+j-1], dp_table[i, j])
            elif j == 0:
                # extend by character from seq1
                if dp_table[i-1, j] and seq1[i-1] == seq3[i+j-1]:
                    dp_table[i, j] = True
                    backpointers[i, j] = (i-1, j, seq1[i-1], 'seq1')
                if debug: print('seq1', i, j, seq1[i-1], '!', seq3[i+j-1], dp_table[i, j])
            else:
                # extend by character from seq2
                if dp_table[i, j-1] and seq2[j-1] == seq3[i+j-1]:
                    dp_table[i, j] = True
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                    if debug: print('seq2', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], dp_table[i, j])                
                # extend by character from seq1
                if dp_table[i-1, j] and seq1[i-1] == seq3[i+j-1]:
                    dp_table[i, j] = True
                    backpointers[i, j] = (i-1, j, seq1[i-1], 'seq1')                
                    if debug: print('seq1', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], dp_table[i, j])
                if not dp_table[i, j]:
                    if debug: print('xxxx', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], dp_table[i, j])

    if return_backpointers or return_table:
        retval = [dp_table[len(seq1), len(seq2)]]
        if return_backpointers:
            retval += [backpointers]
        if return_table:
            retval += [dp_table]
        return tuple(retval)
    else:
        return dp_table[len(seq1), len(seq2)]

In [40]:
def is_interleave_rows(seq1, seq2, seq3, return_backpointers=False, debug=False):
    """Return true if seq3 is some interleaved merge of seq1 and seq2.
    If return_backpointers, also return the set of backpointers to
    reconstruct the interleaving.
    
    This version doesn't build the whole table, just keeps the current and previous rows"""
    
    # dp_table[i, j] is True if first i+j characters of seq is made up of 
    # an interleaving of the first i characters of seq1 and the 
    # first j characters of seq2
    
    if len(seq1) + len(seq2) != len(seq3):
        if return_backpointers:
            retval = [False]
            if return_backpointers:
                retval += [{}]
            return tuple(retval)
        else:
            return False
    

    backpointers = {}

    for i in range(len(seq1)+1):
        row = [False] * (len(seq2)+1)
        for j in range(len(seq2)+1):
            if i == 0 and j == 0:
                row[j] = True
                if debug: print('xxxx', i, j, '!', '!', '!', row[j])
            elif i == 0:
                # extend by character from seq2
                if row[j-1] and seq2[j-1] == seq3[i+j-1]:
                    row[j] = True
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                if debug: print('seq2', i, j, '!', seq2[j-1], seq3[i+j-1], row[j])
            elif j == 0:
                # extend by character from seq1
                if previous_row[j] and seq1[i-1] == seq3[i+j-1]:
                    row[j] = True
                    backpointers[i, j] = (i-1, j, seq1[i-1], 'seq1')
                if debug: print('seq1', i, j, seq1[i-1], '!', seq3[i+j-1], row[j])
            else:
                # extend by character from seq2
                if row[j-1] and seq2[j-1] == seq3[i+j-1]:
                    row[j] = True
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                    if debug: print('seq2', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], row[j])                
                # extend by character from seq1
                if previous_row[j] and seq1[i-1] == seq3[i+j-1]:
                    row[j] = True
                    backpointers[i, j] = (i-1, j, seq1[i-1], 'seq1')                
                    if debug: print('seq1', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], row[j])
                if not row[j]:
                    if debug: print('xxxx', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], row[j])
        previous_row = row

    if return_backpointers:
        retval = [row[-1]]
        if return_backpointers:
            retval += [backpointers]
        return tuple(retval)
    else:
        return row[-1]

In [46]:
def is_interleave_rows2(seq1, seq2, seq3, return_backpointers=False, debug=False):
    """Return true if seq3 is some interleaved merge of seq1 and seq2.
    If return_backpointers, also return the set of backpointers to
    reconstruct the interleaving.
    
    This version doesn't keep the whole table, just the current and previous
    rows. It also builds the current row as it goes along, rather than
    building the whole row and updating elements as required."""
    
    # dp_table[i, j] is True if first i+j characters of seq is made up of 
    # an interleaving of the first i characters of seq1 and the 
    # first j characters of seq2
    
    if len(seq1) + len(seq2) != len(seq3):
        if return_backpointers:
            retval = [False]
            if return_backpointers:
                retval += [{}]
            return tuple(retval)
        else:
            return False
    

    backpointers = {}

    for i in range(len(seq1)+1):
        row = []
        for j in range(len(seq2)+1):
            if i == 0 and j == 0:
                row += [True]
                if debug: print('xxxx', i, j, '!', '!', '!', row[j])
            elif i == 0:
                # extend by character from seq2
                if row[j-1] and seq2[j-1] == seq3[i+j-1]:
                    row += [True]
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                else:
                    row += [False]
                if debug: print('seq2', i, j, '!', seq2[j-1], seq3[i+j-1], row[j])
            elif j == 0:
                # extend by character from seq1
                if previous_row[j] and seq1[i-1] == seq3[i+j-1]:
                    row += [True]
                    backpointers[i, j] = (i-1, j, seq1[i-1], 'seq1')
                else:
                    row += [False]
                if debug: print('seq1', i, j, seq1[i-1], '!', seq3[i+j-1], row[j])
            else:
                # extend by character from seq2
                if row[j-1] and seq2[j-1] == seq3[i+j-1]:
                    row += [True]
                    backpointers[i, j] = (i, j-1, seq2[j-1], 'seq2')
                    if debug: print('seq2', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], row[j])                
                # extend by character from seq1
                elif previous_row[j] and seq1[i-1] == seq3[i+j-1]:
                    row += [True]
                    backpointers[i, j] = (i-1, j, seq1[i-1], 'seq1')                
                    if debug: print('seq1', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], row[j])
                else:
                    row += [False]
                    if debug: print('xxxx', i, j, seq1[i-1], seq2[j-1], seq3[i+j-1], row[j])
        previous_row = row

    if return_backpointers:
        retval = [row[-1]]
        if return_backpointers:
            retval += [backpointers]
        return tuple(retval)
    else:
        return row[-1]

In [41]:
def show_backtrace_i(bps):
    i = max([0] + [k[0] for k in bps])
    j = max([0] + [k[1] for k in bps])
    chars = ''
    if (i, j) in bps:
        while i != 0 or j != 0:
            if bps[i, j][3] == 'seq1':
                chars += bps[i, j][2].upper()
            else:
                chars += bps[i, j][2]
            i, j = bps[i, j][0], bps[i, j][1] 
        return ''.join(list(reversed(chars)))
    else:
        return ''

In [44]:
[s for s in bills
   if is_interleave(bills[0], bills[1], bills[s])]

[30]

In [41]:
[s for s in bills
   if is_interleave_rows(bills[0], bills[1], bills[s])]

[30]

In [48]:
[s for s in bills
   if is_interleave_rows2(bills[0], bills[1], bills[s])]

[30]

In [45]:
v, bp, t = is_interleave(bills[0], bills[1], bills[30], return_backpointers=True, return_table=True)
print(show_table(t))
print(show_backtrace_i(bp))
v

T . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
T . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
T T . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

True

In [48]:
[s for s in bills
   if is_interleave_recursive(bills[0], bills[1], bills[s])]

[30]

In [70]:
%%timeit
[s for s in bills
   if is_interleave(bills[0], bills[1], bills[s])]

1 loop, best of 3: 2.68 s per loop


In [42]:
%%timeit
[s for s in bills
   if is_interleave_rows(bills[0], bills[1], bills[s])]

1 loop, best of 3: 1.06 s per loop


In [49]:
%%timeit
[s for s in bills
   if is_interleave_rows2(bills[0], bills[1], bills[s])]

1 loop, best of 3: 1.56 s per loop


In [71]:
%%timeit
[s for s in bills
   if is_interleave_recursive(bills[0], bills[1], bills[s])]

1 loop, best of 3: 660 ms per loop


# Sense solution
Took 
* 22.9 seconds to load file
* 14906.8 seconds to find subsequences (4.14 hours; 4 hours, 8 minutes, 26.8 seconds)
* 41726.9 seconds to check all interleavings (11.59 hours; 11 hours, 35 minutes, 26.9 seconds)

Total of 15 hours 44 minutes 16.6 seconds.

In [19]:
rtime = 14906.8
(rtime / 3600,
 int(rtime / 3600), 
 int(rtime / 60 - int(rtime / 3600) * 60), 
 rtime - int(rtime / 60) * 60
)

(4.140777777777777, 4, 8, 26.799999999999272)

In [20]:
rtime = 41726.9 
(rtime / 3600,
 int(rtime / 3600), 
 int(rtime / 60 - int(rtime / 3600) * 60), 
 rtime - int(rtime / 60) * 60
)

(11.590805555555557, 11, 35, 26.900000000001455)

In [21]:
rtime = 22.9 + 14906.8 + 41726.9 
(rtime / 3600,
 int(rtime / 3600), 
 int(rtime / 60 - int(rtime / 3600) * 60), 
 rtime - int(rtime / 60) * 60
)

(15.737944444444445, 15, 44, 16.599999999998545)